In [1]:
import pandas as pd

In [2]:
all_weeks = pd.date_range(start='2023-07-10', end='2025-03-01', freq='W-MON')

weekly_df = pd.DataFrame({
    'WEEK_START': all_weeks
})

final_metrics = weekly_df.copy()

In [3]:
acled = pd.read_csv('acled_all_countries.csv')
acled = acled[acled.actor1 == 'Military Forces of Israel (2022-)']
acled['DATE'] = pd.to_datetime(acled['event_date'])
acled['WEEK_START'] = acled['DATE'].dt.to_period('W').dt.start_time

weekly_deaths = acled.groupby('WEEK_START')['fatalities'].sum()
deaths_df = pd.DataFrame(weekly_deaths).reset_index()
deaths_df = deaths_df.rename(columns={'fatalities' : 'escalation'})
final_metrics = final_metrics.merge(deaths_df, how='left')

In [4]:
hostages = pd.read_csv('gdelt_hostages.csv')
hostages['DATE'] = pd.to_datetime(hostages['SQLDATE'], format='%Y%m%d')
hostages['WEEK_START'] = hostages['DATE'].dt.to_period('W').dt.start_time
filtered = hostages[hostages['EventCode'].isin([841])].copy()

hostages_weekly = (
    filtered
    .groupby(['WEEK_START', 'EventCode'])
    .size()
    .unstack(fill_value=0)
    .rename(columns={841: 'return_hostages'})
    .reset_index()
)

final_metrics = final_metrics.merge(hostages_weekly, how='left')

In [5]:
protests = pd.read_csv('protests.csv')
protests['DATE'] = pd.to_datetime(protests['event_date'])
protests['WEEK_START'] = protests['DATE'].dt.to_period('W').dt.start_time

daily_counts = protests.groupby('WEEK_START').size().reset_index(name='protests')
filtered_daily = daily_counts[daily_counts['protests'] > 25]
final_metrics = final_metrics.merge(filtered_daily, how='left')

In [6]:
quotes = pd.read_csv('df_quotes_final.csv')
quotes['WEEK_START'] = pd.to_datetime(quotes['WEEK_START'])
weekly_score = quotes.groupby('WEEK_START')['score'].mean().reset_index()
final_metrics = final_metrics.merge(weekly_score, how='left')

In [7]:
acled = pd.read_csv('acled.csv')
isr = acled[acled['country'] == 'Israel']
isr['event_date'] = pd.to_datetime(isr['event_date'], dayfirst=True, errors='coerce')
isr['WEEK_START'] = isr['event_date'] - pd.to_timedelta(isr['event_date'].dt.weekday, unit='d')
weekly_isr_deaths = isr.groupby('WEEK_START', as_index=False)['fatalities'].sum()

final_metrics = final_metrics.merge(weekly_isr_deaths, how='left')

/var/folders/2z/4ccy9jvd5nl93399x6xrxyd80000gn/T/ipykernel_62651/3766262022.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isr['event_date'] = pd.to_datetime(isr['event_date'], dayfirst=True, errors='coerce')
/var/folders/2z/4ccy9jvd5nl93399x6xrxyd80000gn/T/ipykernel_62651/3766262022.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isr['WEEK_START'] = isr['event_date'] - pd.to_timedelta(isr['event_date'].dt.weekday, unit='d')


In [8]:
final_metrics = final_metrics.fillna(0)
final_metrics = final_metrics.rename(columns={'score' : 'rhetoric_position', 'fatalities' : 'fatalities_isr'})
final_metrics

,WEEK_START,escalation,return_hostages,protests,rhetoric_position,fatalities_isr
0,2023-07-10,0,1.0,95.0,0.000000,0.0
1,2023-07-17,1,0.0,144.0,0.000000,0.0
2,2023-07-24,0,0.0,67.0,0.000000,0.0
3,2023-07-31,0,1.0,31.0,0.000000,2.0
4,2023-08-07,0,0.0,58.0,-1.000000,0.0
...,...,...,...,...,...,...
81,2025-01-27,37,231.0,0.0,0.000000,0.0
82,2025-02-03,42,188.0,0.0,0.125000,0.0
83,2025-02-10,41,288.0,0.0,0.666667,0.0
84,2025-02-17,45,223.0,0.0,1.000000,0.0


In [9]:
final_metrics.to_csv('variables.csv')